In [1]:
# Import Libaries and Packages
import numpy as np
import pandas as pd
import re

## Clean Articles Final

This noteboks performs the following procedures:
    
    1. Reads in extracted and cleanded data from prior processes as a csv. 
    2. Performs addtional preprocessing.
    3. Discards data that does not contain selected keywords.

In [17]:
articles_raw = pd.read_csv("rawData.csv") #Do not run more than once - hogs memory and takes some time
articles_raw.head()

,author,title,publisher,source_url,timeStamp,description
0,David Hundeyin,Crypto Mining Giant Bitmain Acquires Bitcoin C...,Crypto Coins News,https://www.ccn.com/crypto-mining-giant-bitmai...,2018-10-05 23:25:23+00:00,Open source browser-based cryptocurrency walle...
1,Chloe Aiello,"Bitcoin is close to bottoming, cryptocurrency ...",CNBC,https://www.cnbc.com/2018/10/05/bitcoin-is-clo...,2018-10-05 23:03:00+00:00,"Bitcoin is close to bottoming, said Spencer Bo..."
2,Lawrence Abrams,The Week in Ransomware - October 5th 2018 - Re...,Bleepingcomputer.com,https://www.bleepingcomputer.com/news/security...,2018-10-05 23:02:12+00:00,Very very quiet week this. Not much new ransom...
3,Osato Avan-Nomayo,Institutional Investors Bullish About Cryptocu...,Bitcoinist.com,https://bitcoinist.com/institutional-investors...,2018-10-05 23:00:03+00:00,A recent survey by Fundstrat Global Advisors h...
4,Vildana Hajric,"With its volatility on the decline, is Bitcoin...",Latimes.com,http://www.latimes.com/business/la-fi-bitcoin-...,2018-10-05 22:55:00+00:00,Is this the start of a new phase for Bitcoin o...


In [18]:
articles_raw.tail()

,author,title,publisher,source_url,timeStamp,description
976,Marguerite Reardon,California's 'gold standard' net neutrality be...,Cnet.com,https://www.cnet.com/news/californias-gold-sta...,2018-10-01 00:27:22+00:00,It's official. California has adopted net neut...
977,Cali Haan,Permalink to Port of San Diego Hit by “Serious...,Crowdfundinsider.com,https://www.crowdfundinsider.com/2018/09/13960...,2018-10-01 00:15:00+00:00,Port of San Diego CEO Randa Coniglio announced...
978,Katie Richards,Top Marketers Explain How Emerging Tech Will T...,Adweek.com,http://www.adweek.com/brand-marketing/top-mark...,2018-10-01 00:00:51+00:00,"Year-to-year, week-to-week, even day-to-day te..."
979,Sys-con.com,"Notification: OCTCoin, from OCT Corp., to Be L...",Sys-con.com,http://news.sys-con.com/node/4324481,2018-10-01 00:00:02+00:00,"OCT Corp., headquartered in Yokohama, Japan, Y..."
980,Sys-con.com,"Notification: OCTCoin, from OCT Corp., to Be L...",Sys-con.com,http://businesswire.sys-con.com/node/4324481,2018-10-01 00:00:02+00:00,"OCT Corp., headquartered in Yokohama, Japan, Y..."


In [19]:
articles_raw.describe()

,author,title,publisher,source_url,timeStamp,description
count,970,981,981,981,981,978
unique,554,891,304,978,944,916
top,Sys-con.com,Aid groups now using the same data technology ...,Newsbtc.com,https://www.investopedia.com/tech/tron-ceo-jus...,2018-10-04 11:39:12+00:00,This is a paid-for submitted press release. CC...
freq,50,11,81,2,6,13


__Preprocess and Clean Data__

In [20]:
#Reset index
articles_raw = articles_raw.reset_index()
articles_raw = articles_raw.drop('index', axis = 1)
articles_raw.head()

,author,title,publisher,source_url,timeStamp,description
0,David Hundeyin,Crypto Mining Giant Bitmain Acquires Bitcoin C...,Crypto Coins News,https://www.ccn.com/crypto-mining-giant-bitmai...,2018-10-05 23:25:23+00:00,Open source browser-based cryptocurrency walle...
1,Chloe Aiello,"Bitcoin is close to bottoming, cryptocurrency ...",CNBC,https://www.cnbc.com/2018/10/05/bitcoin-is-clo...,2018-10-05 23:03:00+00:00,"Bitcoin is close to bottoming, said Spencer Bo..."
2,Lawrence Abrams,The Week in Ransomware - October 5th 2018 - Re...,Bleepingcomputer.com,https://www.bleepingcomputer.com/news/security...,2018-10-05 23:02:12+00:00,Very very quiet week this. Not much new ransom...
3,Osato Avan-Nomayo,Institutional Investors Bullish About Cryptocu...,Bitcoinist.com,https://bitcoinist.com/institutional-investors...,2018-10-05 23:00:03+00:00,A recent survey by Fundstrat Global Advisors h...
4,Vildana Hajric,"With its volatility on the decline, is Bitcoin...",Latimes.com,http://www.latimes.com/business/la-fi-bitcoin-...,2018-10-05 22:55:00+00:00,Is this the start of a new phase for Bitcoin o...


#### Methodology: 
We have a list of keywords (defined below) that we deem relevant to Bitcoin trading and price. If an article does not mention one of these keywords in the first 300 characters (heuristic/empirical analysis of where the lede of an article ends) we deem it irrelevant.

In [22]:
#Sample of first 300 characters of the first 30 articles. 
#As you can see, relevant articles will mention Bitcoin or a keyword in this selection.
for article in articles_raw['description'][:30]:
    print(article)

Open source browser-based cryptocurrency wallet Telescope has officially announced its acquisition by Bitmain Technologies Inc., the world’s largest crypto mining rig manufacturer, which also runs one of the world’s most extensive cryptocurrency mining pools.…
Bitcoin is close to bottoming, said Spencer Bogart of cryptocurrency and blockchain venture firm Blockchain Capital.
Very very quiet week this. Not much new ransomware to report and only released of well known variants like Matrix, Unlock92, and Dharma ransomware infections. The biggest news was the shut down of numerous restaurants that are part of the Recipe Unlimited gro…
A recent survey by Fundstrat Global Advisors has revealed bullish sentiments among institutional investors in regards to the prospects of cryptocurrency performance during an economic recession. Investors Say a Recession Will Boost Cryptocurrency Prices Accor…
Is this the start of a new phase for Bitcoin or the beginning of the end for the biggest cryptocurre

__Filtering Keyword List__

In [30]:
#Define list of keywords and new 
keywords = pd.read_csv("corpora.csv")
keywords=keywords.Words
articles_final = articles_raw


In [31]:
articles_final.head()

,author,title,publisher,source_url,timeStamp,description
0,David Hundeyin,Crypto Mining Giant Bitmain Acquires Bitcoin C...,Crypto Coins News,https://www.ccn.com/crypto-mining-giant-bitmai...,2018-10-05 23:25:23+00:00,Open source browser-based cryptocurrency walle...
1,Chloe Aiello,"Bitcoin is close to bottoming, cryptocurrency ...",CNBC,https://www.cnbc.com/2018/10/05/bitcoin-is-clo...,2018-10-05 23:03:00+00:00,"Bitcoin is close to bottoming, said Spencer Bo..."
2,Lawrence Abrams,The Week in Ransomware - October 5th 2018 - Re...,Bleepingcomputer.com,https://www.bleepingcomputer.com/news/security...,2018-10-05 23:02:12+00:00,Very very quiet week this. Not much new ransom...
3,Osato Avan-Nomayo,Institutional Investors Bullish About Cryptocu...,Bitcoinist.com,https://bitcoinist.com/institutional-investors...,2018-10-05 23:00:03+00:00,A recent survey by Fundstrat Global Advisors h...
4,Vildana Hajric,"With its volatility on the decline, is Bitcoin...",Latimes.com,http://www.latimes.com/business/la-fi-bitcoin-...,2018-10-05 22:55:00+00:00,Is this the start of a new phase for Bitcoin o...


In [32]:
#Traverse the articles array. 
#If you find an article that does not contain a keyword in the first 300 characters, remove it.
n_removed = 0
indices_to_drop = []

for i in articles_final.index:
    first300 = articles_final['description'][i] #Grab first 300 characters of article text
    title = articles_final['title'][i]
    n_keywords = 0
    try:
        assert type(first300) is str
        assert type(title) is str
    except:
        print("Found error in article", i, "- Passing")
        indices_to_drop.append(i)
        continue
    
    for keyword in keywords:
        if keyword in first300 or keyword in title:
            n_keywords += 1
            break
        else:
            pass
    
    if n_keywords == 0:
        indices_to_drop.append(i)
        
    if len(indices_to_drop) % 100 == 0:
        print("Marked", len(indices_to_drop), "articles for removal")
    
    if i % 10 == 0:
        print("Article", i)
        
print("Total:", len(indices_to_drop), "articles marked for removal.")

Marked 0 articles for removal
Article 0
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Article 10
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Marked 0 articles for removal
Article 20
Article 30
Article 40
Article 50
Article 60
Found error in article 67 - Passing
Article 70
Article 80
Article 90
Article 100
Article 110
Article 120
Article 130
Article 140
Article 150
Article 160
Article 170
Article 180
Article 190
Article 200
Article 210
Article 220
Article 230
Article 240
Article 250
Article 260
Article 270
Article 280
Found error in article 289 - Passing
Article 290
Article 

In [33]:
articles_final = articles_final.drop(indices_to_drop)

In [34]:
articles_final = articles_final.reset_index()
articles_final = articles_final.drop('index', axis = 1)
articles_final.head()

,author,title,publisher,source_url,timeStamp,description
0,David Hundeyin,Crypto Mining Giant Bitmain Acquires Bitcoin C...,Crypto Coins News,https://www.ccn.com/crypto-mining-giant-bitmai...,2018-10-05 23:25:23+00:00,Open source browser-based cryptocurrency walle...
1,Chloe Aiello,"Bitcoin is close to bottoming, cryptocurrency ...",CNBC,https://www.cnbc.com/2018/10/05/bitcoin-is-clo...,2018-10-05 23:03:00+00:00,"Bitcoin is close to bottoming, said Spencer Bo..."
2,Lawrence Abrams,The Week in Ransomware - October 5th 2018 - Re...,Bleepingcomputer.com,https://www.bleepingcomputer.com/news/security...,2018-10-05 23:02:12+00:00,Very very quiet week this. Not much new ransom...
3,Osato Avan-Nomayo,Institutional Investors Bullish About Cryptocu...,Bitcoinist.com,https://bitcoinist.com/institutional-investors...,2018-10-05 23:00:03+00:00,A recent survey by Fundstrat Global Advisors h...
4,Vildana Hajric,"With its volatility on the decline, is Bitcoin...",Latimes.com,http://www.latimes.com/business/la-fi-bitcoin-...,2018-10-05 22:55:00+00:00,Is this the start of a new phase for Bitcoin o...


In [36]:
articles_final.to_csv("final_articles_cj.csv")

___